In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pdfplumber
!pip install langdetect
!pip install PyPDF2
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.3 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""ExtraccionQuechuaCollao.ipynb

Se instalan las librerias que se usaran
"""

import os
import re
import nltk
import pdfplumber
from langdetect import detect
from google.colab import drive
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

"""Se crea un acceso a Drive para acceder a los archivos"""

drive.mount('/content/drive')

# Descargar recursos adicionales de NLTK
nltk.download('punkt')
nltk.download('stopwords')

"""Se define la funcion detect_language para detectar aquellas oraciones en español dentro de los documentos"""

def detect_language(text):
  try:
    idioma = detect(text)
    return idioma
  except:
    return None

"""Se define la función extract_text_from_pdf para extraer el texto de los documentos y filtrar solo aquellas oraciones en quechua"""

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    oraciones = text.split('.')
    oraciones = [oracion.strip() for oracion in oraciones if oracion.strip()]
    texto_quechua=[]
    for oracion in oraciones:
      idioma = detect_language(oracion)
      if idioma != "es":
          texto_quechua.append(oracion)
    return " ".join(texto_quechua)

#def clean_text(text):
    # Eliminar caracteres especiales y números
    #text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Convertir a minúsculas
    # text = text.lower()
    # Tokenización
    # tokens = word_tokenize(text)
    # Eliminar stopwords en español
    # stop_words = set(stopwords.words('spanish'))
    # tokens = [word for word in tokens if word not in stop_words]
    # return " ".join(tokens)

"""Se define la función create_corpus que recorre la carpeta en busca de los documentos para crear el corpus"""

def create_corpus(pdf_folder):
    corpus = []
    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            text = extract_text_from_pdf(pdf_path)
            #cleaned_text = clean_text(text)
            corpus.append({'documento': text, 'variante': 'collao'})
    return corpus

"""Se define la ruta de la carpeta y se crea el corpus"""

# Ruta al directorio que contiene los archivos PDF
pdf_folder = "/content/drive/My Drive/Corpus/collao"

# Crear el corpus
corpus_collao = create_corpus(pdf_folder)

In [ ]:
import PyPDF2

def extraer_lineas(inicio, fin, pdf_path, tipo):
    paginas = []

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_paginas = len(reader.pages)

        for pagina_num in range(inicio,fin+1):
            pag = []
            pagina = reader.pages[pagina_num]
            contenido = pagina.extract_text()

            # Dividir el contenido de la página en lineas
            palabras = contenido.split('\n')

            #si la linea empieza con digito, entonces se elimina ese digito que es el indicador de pagina
            if palabras[0].isdigit():
              del palabras[0]
            else:
              # si por error la linea empieza con un numero que no deberia ir ahí, lo elimina
              if palabras[0][0].isdigit():
                palabras[0] = palabras[0][1:]

            #une las palabras en una linea
            pag.append(" ".join(palabras))

            #añade las lineas a el texto completo divido por paginas
            paginas.append(" ".join(pag[0].split()))

    #une todo el texto y regresa un diccionario
    return dict([('documento', " ".join(paginas)), ('variante', tipo)])

maychi = extraer_lineas(4,8, "/content/drive/My Drive/Corpus/chanka/maychi.pdf", "Chanka")
chanka5 = extraer_lineas(4,8, "/content/drive/My Drive/Corpus/chanka/antuku.pdf", "Chanka")

In [ ]:
def extraer_lineas(inicio, fin, pdf_path, tipo):
    paginas = []

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_paginas = len(reader.pages)

        for pagina_num in range(inicio,fin+1):
            pag = []
            pagina = reader.pages[pagina_num]
            contenido = pagina.extract_text()

            # Dividir el contenido de la página en lineas
            palabras = contenido.split('\n')
            #si la linea empieza con digito, entonces se elimina ese digito que es el indicador de pagina
            if palabras[0].isdigit():
              del palabras[0]
            else:
              #si la linea es el comienzo de un capitulo, el titulo se guardó al ultimo y se procede a eliminarlo
              del palabras[-5:]

            #une las palabras en una linea
            pag.append(" ".join(palabras))

            #añade las lineas a el texto completo divido por paginas
            paginas.append(" ".join(pag[0].split()))

    #une todo el texto y regresa un diccionario
    return (" ".join(paginas), tipo)
    #return dict([('documento', " ".join(paginas)), ('variante', tipo)])

paginas = extraer_lineas(8,158, "/content/drive/My Drive/Corpus/chanka/Chanka_Extraccion1.pdf", "Chanka")

In [ ]:
def extraer_lineas(inicio, fin, pdf_path, tipo):
    paginas = []

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_paginas = len(reader.pages)

        for pagina_num in range(inicio,fin+1):
            pag = []
            pagina = reader.pages[pagina_num]
            contenido = pagina.extract_text()

            # Dividir el contenido de la página en lineas
            palabras = contenido.split('\n')
            #si la linea empieza con digito, entonces se elimina ese digito que es el indicador de pagina
            if palabras[0].isdigit():
              del palabras[0]

            #une las palabras en una linea
            pag.append(" ".join(palabras))

            #añade las lineas a el texto completo divido por paginas
            paginas.append(" ".join(pag[0].split()))

    #une todo el texto y regresa un diccionario
    return dict([('documento', " ".join(paginas)), ('variante', tipo)])

paginas_2 = extraer_lineas(8,73, "/content/drive/My Drive/Corpus/chanka/saberes.pdf", "Chanka")

In [ ]:
corpusChanka = [
  maychi,
  chanka5,
  paginas,
  paginas_2,
]
len(corpusChanka)

In [ ]:
import re
from pypdf import PdfReader

# Fuente: Urin Qichwa VOCABULARIO PEDAGÓGICO QUECHUA SUREÑO
reader = PdfReader("/content/drive/My Drive/Corpus/unificada/urin.pdf")

# Secciones con oraciones útiles
pages_comm = reader.pages[17:47]  # Comunicación
pages_pers = reader.pages[60:68]  # Personal social y ciencias
pages_camp = reader.pages[109:131]  # Por campos semánticos

document = ""

### Sección Comunicación ###

# Regex patterns
pagination_pttrn = (
    r"Comunicación\nvocabulario\d+\sParte\sI\s-\sQuechua\ssureño"  # Datos de paginación
)
sentences_pttrn = r"\.[A-Za-z]?\s*[A-Za-z]?\s([^.]+\.)\s*‘([^‘’]+)’\."  # Grupo1 oración quechua, grupo2 oración español encerrada en ‘’

# Processing
for page in pages_comm:
    # Clean pagination
    text = re.sub(pagination_pttrn, "", page.extract_text())

    # Find quechua sentences
    matches = re.findall(sentences_pttrn, text)
    for m in matches:
        sentence = m[0].replace("\n", "")
        sentence = sentence.replace("/ ", "/")
        document += sentence + "\n"

### Sección Personal social y ciencias ###

# Regex patterns
pagination_pttrn = r"Personal\ssocial\sy\sCiencia\sy\stecnología\nvocabularioParte\sII\s-\sQuechua sureño\s\d+"  # Datos de paginación
sentences_pttrn = r"\.[A-Za-z]?\s*[A-Za-z]?\s([^.]+\.)\s*‘([^‘’]+)’\."  # Grupo1 oración quechua, grupo2 oración español encerrada en ‘’

# Processing
for page in pages_pers:
    # Clean pagination
    text = re.sub(pagination_pttrn, "", page.extract_text())

    # Find quechua sentences
    matches = re.findall(sentences_pttrn, text)
    for m in matches:
        sentence = m[0].replace("\n", "")
        sentence = sentence.replace("/ ", "/")
        document += sentence + "\n"

### Sección campos semánticos ###

# Regex patterns
pagination_pttrn = r"Palabras\spor\scampos\ssemánticos\sdel\squechua\ssureño\nUsadas\sen\sel\sárea\sde\sPersonal\ssocial\sy\sCiencia\sy\stecnologíaParte\sIII\s-\sQuechua\ssureño\s\d+"  # Datos de paginación
patron = r"([a-z].*?\.)\s*\n\s*((?:[A-Z].*?\.\s*)+)"  # Grupo1 oración en minúscula (palabras), grupo2 grupo de oraciones que empiezan en mayúsucula (ejemplos en quechua)

# Processing
for page in pages_camp:
    # Clean pagination
    text = re.sub(pagination_pttrn, "", page.extract_text())

    # Find quechua sentences
    matches = re.findall(patron, text, re.DOTALL)
    for m in matches:
        sentence = m[1].replace("\n", "")
        sentence = sentence.replace("/ ", "/")
        document += sentence + "\n"


corpusUnificado = {'documento': document, 'variante': 'unificada'}

In [ ]:
len(corpusUnificado)

In [ ]:
corpus=[]
for doc in corpusChanka:
  corpus.append(doc)
corpus.append({'documento': corpusUnificado["documento"], 'variante': corpusUnificado["variante"]})
for doc in corpus_collao:
  corpus.append(doc)

In [ ]:
len(corpus)

In [ ]:
# Guardar el corpus en un archivo de texto
with open("corpus.txt", "w") as f:
    for document in corpus:
        f.write("%s\n" % document)

In [ ]:
print("Número de documentos en el corpus:", len(corpus))

In [ ]:
# Obtener el número de palabras de cada documento y del corpus
num_palabras_documento = []
for documento in corpus:
    num_palabras = len(documento['documento'].split())
    num_palabras_documento.append(num_palabras)
    #print(f"Número de palabras en el documento: {num_palabras}")

# Número total de palabras en el corpus
total_palabras_corpus = sum(num_palabras_documento)
print("Número total de palabras en el corpus:", total_palabras_corpus)


In [ ]:
import matplotlib.pyplot as plt

# Obtener el número de palabras de cada documento y del corpus
num_palabras_documento = []
for documento in corpus:
    num_palabras = len(documento['documento'].split())
    num_palabras_documento.append(num_palabras)

# Número total de palabras en el corpus
total_palabras_corpus = sum(num_palabras_documento)

# Crear gráfico de barras para mostrar el número de palabras en cada documento
plt.figure(figsize=(10, 6))
plt.bar(range(1, len(num_palabras_documento) + 1), num_palabras_documento, color='skyblue')
plt.xlabel('Documento')
plt.ylabel('Número de palabras')
plt.title('Número de palabras en cada documento')
plt.xticks(range(1, len(num_palabras_documento) + 1))

# Agregar el número total de palabras en el gráfico
plt.text(0.5, 0.95, f'Total de palabras: {total_palabras_corpus}', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes, fontsize=15, bbox=dict(facecolor='white', alpha=0.5))


plt.show()